In [1]:
import pandas as pd
import json
import numpy as np
import re
import difflib as kwrd
pd.options.display.max_rows = 500

In [2]:
def similar(org,name):
    org = [y.replace('-',' ').replace('(',' ').replace(')',' ') for y in list(org['name'])]
    name = name.replace('-',' ').replace('(',' ').replace(')',' ')
    match = kwrd.get_close_matches(name,org,n=3, cutoff=0.5) 
    if len(match) > 0:
        return match[0]
    return None

In [3]:
with open ('../backend/dev/resources/files/organisations.json', 'r') as orgs:
    old = json.loads(orgs.read())

In [4]:
old = pd.DataFrame(old)
old['abbr'] = old['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
old['name'] = old['name'].apply(lambda x: x.strip())
old['lname'] = old['name'].apply(lambda x: x.split('(')[0].lower())

In [5]:
new = pd.read_csv('./new_organisation.csv')
new['abbr'] = new['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
new['name'] = new['name'].apply(lambda x: x.strip())
new['lname'] = new['name'].apply(lambda x: x.split('(')[0].lower())
new['review_status'] = 'APPROVED'

In [6]:
old['match'] = old['name'].apply(lambda x: similar(new, x))

In [7]:
URL_REGEX = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
def getUrl(x):
    url = re.findall(URL_REGEX, x)
    if len(url) > 0:
        return "https://" + url[0][0].replace("https://","").replace("http://","")
    return None

In [8]:
merged = pd.merge(new, old, on="lname", how="outer", validate="m:m").replace([np.nan], [None])
merged['name'] = merged.apply(lambda x: x['name_x'] if x['name_x'] else x['name_y'], axis=1)
merged = merged[['id','name','country','country_group','type','url', 'address','program','contribution','expertise','review_status']]
merged['url'] = merged['url'].apply(lambda x: getUrl(x) if x else None)
merged = merged[['id','name','country','country_group','type','url', 'address','program','contribution','expertise','review_status']].sort_values('name')
merged['review_status'] = merged['review_status'].fillna("SUBMITTED")
merged['country_group'] = merged['country_group'].apply(lambda x: None if x == ' ' else x)

In [9]:
merged['address'] = merged['address'].fillna("").str.strip()

In [10]:
merged

,id,name,country,country_group,type,url,address,program,contribution,expertise,review_status
351,None,3M,None,None,None,None,,None,None,None,SUBMITTED
1,2,5 Gyres Institute,United States of America,None,Non-governmental organization,https://www.5gyres.org/,The 5 Gyres Institute\n5792 West Jefferson Bou...,None,None,None,APPROVED
306,306,AD Architektur und Energie,Germany,Europe,Academia and Research,None,An der Wormser Str,ZEEP \n\nZero Emission Energy Port,Reducing levels and impacts of land-based litt...,Energy \n\nEnergy Saving Strategy \n\nZero Emi...,APPROVED
352,None,APEC Ocean and Fisheries Working Group (OFWG),None,None,None,None,,None,None,None,SUBMITTED
168,169,Adult commercial secondary School,Somalia,Africa,Academia and Research,None,"Baladweyn, Somalia",None,None,None,APPROVED
...,...,...,...,...,...,...,...,...,...,...,...
243,243,Youth for our Planet,United Kingdom of Great Britain & Northern Ire...,Europe,Non-governmental organization,https://www.youthforourplanet.com,,Nature restoration advocacy,Reducing levels and impacts of land-based litt...,Environmental Advocacy \nClimate Activism,APPROVED
253,253,ecoSurge,Italy,Europe,Non-governmental organization,https://www.ecosurge.org,"via Murri, Bologna",Impact of covid-19 on plastic through the anal...,Reducing levels and impacts of land-based litt...,"Digital technologies (AI,IoT,data analytics) f...",APPROVED
140,141,mountain2ocean,Germany,Europe,Private Sector,https://www.mountain2ocean.org,"Am Breindlbach 7, Bad Feilnbach, Bavaria, 8307...",1. Lectures: Improving knowledge in students a...,Reduced levels and impacts of land-based litte...,"Marine Biology, Marine Litter",APPROVED
608,None,the Government of Japan,None,None,None,None,,None,None,None,SUBMITTED


In [11]:
new_inserted = merged[merged['id'] != merged['id']].reset_index()
new_inserted['id'] = new_inserted.index + 1000

In [12]:
merged = pd.concat([merged[merged['id'] == merged['id']].sort_values('id'), new_inserted])
merged['id'] = merged['id'].astype('int')

In [13]:
merged.to_csv('./result/organisation_merged.csv', index=False)

In [14]:
unlisted = merged[merged['review_status'] == 'SUBMITTED']
unlisted.to_csv('./result/organisation_unlisted.csv')

In [15]:
merged.to_json('../backend/dev/resources/files/organisations_new.json', orient='records')